In [77]:
import pandas as pd
from sodapy import Socrata
import geopy
import time
from time import sleep

In [78]:
#API Would not be the best option due to the import limitation.
#client = Socrata("data.cityofnewyork.us", None)
#results = client.get("43nn-pn8j", limit=2000)

In [79]:
#Reading csv file
restaurant_df = pd.read_csv('./restaurant.csv')

In [80]:
#Information of columns and dtype.
restaurant_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218563 entries, 0 to 218562
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  218563 non-null  int64  
 1   DBA                    218009 non-null  object 
 2   BORO                   218563 non-null  object 
 3   BUILDING               218122 non-null  object 
 4   STREET                 218554 non-null  object 
 5   ZIPCODE                215818 non-null  float64
 6   PHONE                  218561 non-null  object 
 7   CUISINE DESCRIPTION    216230 non-null  object 
 8   INSPECTION DATE        218563 non-null  object 
 9   ACTION                 216230 non-null  object 
 10  VIOLATION CODE         215095 non-null  object 
 11  VIOLATION DESCRIPTION  215095 non-null  object 
 12  CRITICAL FLAG          218563 non-null  object 
 13  SCORE                  208075 non-null  float64
 14  GRADE                  106854 non-nu

In [81]:
#Remove rows that doesn't have names.
restaurant_df = restaurant_df[restaurant_df['DBA'].notna()]

In [82]:
#Checking above has completed
print(len(restaurant_df.index))

218009


In [90]:
#Filtering only "Manhattan" boro
restaurant_df = restaurant_df.loc[(restaurant_df['BORO'] == 'Manhattan')]
restaurant_df = restaurant_df.reset_index(drop=True)

In [91]:
#Checking above has completed
print(len(restaurant_df.index))
restaurant_df.head()

80708


,DBA,BORO,BUILDING,STREET,ZIPCODE,Latitude,Longitude
0,GRAND & ESSEX,Manhattan,359,GRAND STREET,10002.0,40.716639,-73.988998
1,THE WOLFE,Manhattan,425,AMSTERDAM AVENUE,10024.0,40.784048,-73.977651
2,ROMA PIZZA,Manhattan,465,6 AVENUE,10011.0,40.735485,-73.998214
3,"SHRINE, YATENGA",Manhattan,22712269,7 AVENUE,NaN,0.000000,0.000000
4,AVIATOR GRILL AT INTREPID,Manhattan,1,INTREPID SQ,NaN,0.000000,0.000000


In [85]:
#Keeping necessary information only
#DBA, BORO, Building, Street, Zipcode, Phone, Latitude, and Longitude

restaurant_df = restaurant_df[['DBA','BORO','BUILDING', 'STREET', 'ZIPCODE', 'Latitude', 'Longitude']]


In [86]:
#Displaying updated dataframe
restaurant_df.head()

,DBA,BORO,BUILDING,STREET,ZIPCODE,Latitude,Longitude
0,GRAND & ESSEX,Manhattan,359,GRAND STREET,10002.0,40.716639,-73.988998
12,THE WOLFE,Manhattan,425,AMSTERDAM AVENUE,10024.0,40.784048,-73.977651
13,ROMA PIZZA,Manhattan,465,6 AVENUE,10011.0,40.735485,-73.998214
28,"SHRINE, YATENGA",Manhattan,22712269,7 AVENUE,NaN,0.000000,0.000000
30,AVIATOR GRILL AT INTREPID,Manhattan,1,INTREPID SQ,NaN,0.000000,0.000000


In [87]:
#Checking if there is any null value
restaurant_df.isnull().any()

DBA          False
BORO         False
BUILDING      True
STREET       False
ZIPCODE       True
Latitude      True
Longitude     True
dtype: bool

In [93]:
#Adding Address to add ZIPCODE
from geopy.geocoders import Nominatim

geolocator = geopy.Nominatim(user_agent='http')

restaurant_df["ADRESS"] = restaurant_df['BUILDING'] + ' ' + restaurant_df['STREET'] + ', ' + restaurant_df["BORO"]

restaurant_df.head()

,DBA,BORO,BUILDING,STREET,ZIPCODE,Latitude,Longitude,ADRESS
0,GRAND & ESSEX,Manhattan,359,GRAND STREET,10002.0,40.716639,-73.988998,"359 GRAND STREET, Manhattan"
1,THE WOLFE,Manhattan,425,AMSTERDAM AVENUE,10024.0,40.784048,-73.977651,"425 AMSTERDAM AVENUE, Manhattan"
2,ROMA PIZZA,Manhattan,465,6 AVENUE,10011.0,40.735485,-73.998214,"465 6 AVENUE, Manhattan"
3,"SHRINE, YATENGA",Manhattan,22712269,7 AVENUE,NaN,0.000000,0.000000,"22712269 7 AVENUE, Manhattan"
4,AVIATOR GRILL AT INTREPID,Manhattan,1,INTREPID SQ,NaN,0.000000,0.000000,"1 INTREPID SQ, Manhattan"


In [169]:
def get_zipcode(address):
    location = geolocator.geocode(address, timeout=None)
    #data = location.raw
    #loc_data = data['display_name'].split(", ")
    #zipcode = loc_data[-2]
    time.sleep(1)
    return location

In [170]:
get_zipcode("2935 Veterans Road, Staten Island")

Location(M&T Bank, 2935, Veterans Road West, Staten Island, Richmond County, City of New York, New York, 10309, United States, (40.5265539, -74.2308288, 0.0))

In [171]:
nan_rows['zipcode'] = nan_rows.apply(get_zipcode(nan_rows['address']))

AssertionError: 